In [2]:
from jupyter_dash import JupyterDash  
import dash
import plotly.graph_objects as go
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.express as px
from dash.dependencies import Input, Output
# from fbprophet import Prophet
from datetime import datetime,timedelta,date
from clickhouse_driver import connect
import clickhouse_driver
import datetime as dList

In [3]:
print(dList.date.today())

2021-03-31


In [9]:
now_begin_date = dList.date.today()
now_begin_date = str(now_begin_date)[0:8]+'01'
now_begin_date

'2021-03-01'

In [ ]:
JupyterDash.infer_jupyter_proxy_config()

In [5]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

server = app.server

#获取两个日期间的所有日期
def getEveryDay(begin_date,end_date):
    date_list = []
    begin_date = dList.datetime.strptime(str(begin_date), "%Y-%m-%d")
    end_date = dList.datetime.strptime(str(end_date),"%Y-%m-%d")
    while begin_date <= end_date:
        date_str = begin_date.strftime("%Y-%m-%d")
        date_list.append(date_str)
        begin_date += dList.timedelta(days=1)
    return date_list

#求预期的虚线 1个月
def estimate_retention(this_start_month,this_end_month,this_month_target):
#     delta = dList.datetime.strptime(this_end_month,"%Y-%m-%d").day - dList.datetime.strptime(this_start_month,"%Y-%m-%d").day +1
    delta = dList.datetime.strptime(str(this_end_month),"%Y-%m-%d").day - dList.datetime.strptime(str(this_start_month),"%Y-%m-%d").day +1
    k = this_month_target / delta
    b = -k
    this_all_date = getEveryDay(this_start_month,this_end_month)

    estList = []
    for i in range(0,delta+1):
        estX = i
        estY = round(k*(estX))
        if i!= 0:
            estList.append((datetime.strptime(this_all_date[i-1],"%Y-%m-%d"),estY))
#     print(estList)
    pd.DataFrame(estList, columns=['ds', 'ret']).to_csv('target_ret.csv',index=False)


In [6]:
#求预期的虚线 2个月
def estimate_retention_2month(this_start_month,this_end_month,this_month_target):
#     delta = dList.datetime.strptime(this_end_month,"%Y-%m-%d").day - dList.datetime.strptime(this_start_month,"%Y-%m-%d").day +1
    delta = (dList.datetime.strptime(str(this_end_month),"%Y-%m-%d") - dList.datetime.strptime(str(this_start_month),"%Y-%m-%d")).days +1
    #print(delta)
    k = this_month_target / delta
    b = -k
    this_all_date = getEveryDay(this_start_month,this_end_month)

    estList = []
    for i in range(0,delta+1):
        estX = i
        estY = round(k*(estX))
        if i!= 0:
            estList.append((datetime.strptime(this_all_date[i-1],"%Y-%m-%d"),estY))
#     print(estList)
    pd.DataFrame(estList, columns=['ds', 'ret']).to_csv('target_ret_2month.csv',index=False)   

In [7]:
#求预期的虚线 3个月
def estimate_retention_3month(this_start_month,this_end_month,this_month_target):
#     delta = dList.datetime.strptime(this_end_month,"%Y-%m-%d").day - dList.datetime.strptime(this_start_month,"%Y-%m-%d").day +1
    delta = (dList.datetime.strptime(str(this_end_month),"%Y-%m-%d") - dList.datetime.strptime(str(this_start_month),"%Y-%m-%d")).days +1
    #print(delta)
    k = this_month_target / delta
    b = -k
    this_all_date = getEveryDay(this_start_month,this_end_month)

    estList = []
    for i in range(0,delta+1):
        estX = i
        estY = round(k*(estX))
        if i!= 0:
            estList.append((datetime.strptime(this_all_date[i-1],"%Y-%m-%d"),estY))
#     print(estList)
    pd.DataFrame(estList, columns=['ds', 'ret']).to_csv('target_ret_3month.csv',index=False)    

In [8]:
def get_data(begin_date,j,branch="所有支行"):
    client = clickhouse_driver.Client(host='chi-ftabc-clickhouse-0-0.default.svc.cluster.local', port='9000', database='ftabcch', user='ftabc', password='aihub@2020')
    conn = connect(host='chi-ftabc-clickhouse-3-0.default.svc.cluster.local', port='9000', database='ftabcch', user='ftabc', password='aihub@2021')
    cursor = conn.cursor()
    cursor.execute('''select addDays(toDate('{}'),{}),toStartOfMonth(addDays(toDate('{}'),{}))'''.format(begin_date,j,begin_date,j))
    this_date,this_month = cursor.fetchall()[0]#[0][0]-timedelta(days=1)
    #print(this_date,this_month)
    
####拿到当前月份、上月份、上上月份 的 第一天 和 最后一天
    cursor.execute('''
        select 
            '{}',
            addDays( toStartOfMonth( addDays(toDate('{}'),31) ),-1 ),
            toStartOfMonth( addDays(toStartOfMonth(toDate('{}')),-1) ),
            addDays(toStartOfMonth(toDate('{}')),-1),
            toStartOfMonth( addDays(toStartOfMonth( addDays(toStartOfMonth(toDate('{}')),-1) ),-1) ),
            addDays(toStartOfMonth( addDays(toStartOfMonth(toDate('{}')),-1) ),-1),
            toStartOfMonth( addDays(toDate('{}'),31) ),
            addDays( toStartOfMonth( addDays(toStartOfMonth( addDays(toDate('{}'),31) ),31) ),-1 )
    '''.format(this_month,this_month,this_month,this_month,this_month,this_month,this_month,this_month))
    this_two_month = cursor.fetchall()
    this_start_month = this_two_month[0][0]
    this_end_month = this_two_month[0][1]
    this_l1_start_month = this_two_month[0][2]
    this_l1_end_month = this_two_month[0][3]
    this_l2_start_month = this_two_month[0][4]
    this_l2_end_month = this_two_month[0][5]
    this_n1_start_month = this_two_month[0][6]
    this_n1_end_month = this_two_month[0][7]    
    # ####拿到当前月份、上月份、上上月份
#     print(this_start_month) #2020-11-01
#     print(this_end_month) #2020-11-30
#     print(this_l1_start_month) #2020-10-01
#     print(this_l1_end_month) #2020-10-31
#     print(this_l2_start_month.strftime('%Y-%m-%d')) # 2020-09-01
#     print(this_l2_end_month.strftime('%Y-%m-%d')) #2020-09-30
    
    #print(getEveryDay(str(this_start_month),str(this_end_month)))       
                                                                                                
    
    ####上上一月的新用户计算 9
    if(branch=="所有支行"):
        cursor.execute('''
            select distinct id from
            (select distinct id,value from ads_user_tag_distribute
                    where tag_id = 15 and value like '{}%') as reg
            where reg.value BETWEEN '{}' and '{}' and length(reg.value)=10
            order by value
        '''.format(this_l2_start_month.strftime('%Y-%m-%d')[0:5],this_l2_start_month,this_l2_end_month))
    new_user_l2 = cursor.fetchall()
    #print(this_l2_start_month.strftime('%Y-%m-%d')[5:7],"月新用户",len(new_user_l2))
    
    
    ####上一月的新用户计算 10
    cursor.execute('''
        select count(distinct id) from
        (select distinct id,value from ads_user_tag_distribute
                where tag_id = 15 and value like '{}%') as reg
        where reg.value BETWEEN '{}' and '{}' and length(reg.value)=10
    '''.format(this_l1_start_month.strftime('%Y-%m-%d')[0:5],this_l1_start_month,this_l1_end_month))
    new_user_l1 = cursor.fetchall()[0][0]
    #print(this_l1_start_month.strftime('%Y-%m-%d')[5:7],"月新用户",new_user_l1)
    
    ####上一月的留存用户计算 9
    cursor.execute('''
        select count(distinct id) from(
            select distinct id from dws_event_2020_middle_distribute
                where app_date BETWEEN  '{}' and '{}' and
                id GLOBAL IN (
                    select distinct id from
                    (select distinct id,value from ads_user_tag_distribute
                            where tag_id = 15 and value BETWEEN '{}' and '{}' and length(value)=10
                            )
                ) 
            ) 
    '''.format(this_l1_start_month,this_l1_end_month,this_l2_start_month,this_l2_end_month))
    this_retained = cursor.fetchall()[0][0]
    
    #上月的1月留存率
    if(len(new_user_l2) == 0):
        Retention_rates = 0
    else:
        Retention_rates = this_retained/len(new_user_l2) 
    #本月的目标留存数
    this_month_target = round(new_user_l1*Retention_rates)
    
    
    ####本日的1月内留存用户计算 
    ####eg:10月注册的在11月里任意天操作过都算留存 
    cursor.execute('''
        select count(distinct id) from(
            select distinct id from dws_event_2020_middle_distribute
                where app_date BETWEEN  '{}' and '{}' and
                id GLOBAL IN (
                    select distinct id from
                    (select id from ads_user_tag_distribute
                            where tag_id = 15 and value BETWEEN '{}' and '{}' and length(value)=10
                            )
                )
            )  
    '''.format(this_month,this_date,this_l1_start_month,this_l1_end_month))
    this_retained = cursor.fetchall()[0][0]
    if this_date == this_month :
        pd.DataFrame([(this_date,this_retained)],columns=['ds','y']).to_csv('currrent_ret.csv',index=False)
    else :
        pd.DataFrame([(this_date,this_retained)],columns=['ds','y']).to_csv('currrent_ret.csv',index=False,mode='a',header=False)    
    
    estimate_retention(this_start_month,this_end_month,this_month_target)
    cursor.close()
    conn.close()
    if this_month_target != 0:
        return new_user_l1,this_retained,this_month_target,round((this_retained/this_month_target)*100,2)
    else :
        return new_user_l1,this_retained,this_month_target,0    
    

In [9]:
def get_data_2month(begin_date,j,branch="所有支行"):
    client = clickhouse_driver.Client(host='chi-ftabc-clickhouse-0-0.default.svc.cluster.local', port='9000', database='ftabcch', user='ftabc', password='aihub@2020')
    conn = connect(host='chi-ftabc-clickhouse-3-0.default.svc.cluster.local', port='9000', database='ftabcch', user='ftabc', password='aihub@2021')
    cursor = conn.cursor()
    cursor.execute('''select addDays(toDate('{}'),{}),toStartOfMonth(addDays(toDate('{}'),{}))'''.format(begin_date,j,begin_date,j))
    this_date,this_month = cursor.fetchall()[0]#[0][0]-timedelta(days=1)
    #print(this_date,this_month)
    delta_month = dList.datetime.strptime(str(this_month),"%Y-%m-%d").month - dList.datetime.strptime(str(begin_date),"%Y-%m-%d").month +12*(dList.datetime.strptime(str(this_month),"%Y-%m-%d").year - dList.datetime.strptime(str(begin_date),"%Y-%m-%d").year)
    print(delta_month,this_date)
    if(delta_month>0):
        if(delta_month%2==1):
            cursor.execute('''select toStartOfMonth( addDays(toStartOfMonth(toDate('{}')),-1) )'''.format(this_month))
            this_month = cursor.fetchall()[0][0]
            print(this_month)

                
####拿到当前月份、上月份、上上月份 的 第一天 和 最后一天
    cursor.execute('''
        select 
            '{}',
            addDays( toStartOfMonth( addDays(toDate('{}'),31) ),-1 ),
            toStartOfMonth( addDays(toStartOfMonth(toDate('{}')),-1) ),
            addDays(toStartOfMonth(toDate('{}')),-1),
            toStartOfMonth( addDays(toStartOfMonth( addDays(toStartOfMonth(toDate('{}')),-1) ),-1) ),
            addDays(toStartOfMonth( addDays(toStartOfMonth(toDate('{}')),-1) ),-1),
            toStartOfMonth( addDays(toDate('{}'),31) ),
            addDays( toStartOfMonth( addDays(toStartOfMonth( addDays(toDate('{}'),31) ),31) ),-1 )
    '''.format(this_month,this_month,this_month,this_month,this_month,this_month,this_month,this_month))
    this_two_month = cursor.fetchall()
    this_start_month = this_two_month[0][0] #2020-11-01
    this_end_month = this_two_month[0][1] #2020-11-30
    this_l1_start_month = this_two_month[0][2] #2020-10-01
    this_l1_end_month = this_two_month[0][3] #2020-10-31
    this_l2_start_month = this_two_month[0][4] # 2020-09-01
    this_l2_end_month = this_two_month[0][5] #2020-09-30
    this_n1_start_month = this_two_month[0][6] #2020-12-01
    this_n1_end_month = this_two_month[0][7]   #2020-12-31
    # ####拿到当前月份、上月份、上上月份
#     print(this_start_month) #2020-11-01
#     print(this_end_month) #2020-11-30
#     print(this_l1_start_month) #2020-10-01
#     print(this_l1_end_month) #2020-10-31
#     print(this_l2_start_month.strftime('%Y-%m-%d')) # 2020-09-01
#     print(this_l2_end_month.strftime('%Y-%m-%d')) #2020-09-30
 
                                                                                                
    
    ####上上一月的新用户计算 9
    if(branch=="所有支行"):
        cursor.execute('''
            select distinct id from
            (select distinct id,value from ads_user_tag_distribute
                    where tag_id = 15 and value like '{}%') as reg
            where reg.value BETWEEN '{}' and '{}' and length(reg.value)=10
            order by value
        '''.format(this_l2_start_month.strftime('%Y-%m-%d')[0:5],this_l2_start_month,this_l2_end_month))
    new_user_l2 = cursor.fetchall()
    #print(this_l2_start_month.strftime('%Y-%m-%d')[5:7],"月新用户",len(new_user_l2))
    
    
    ####上一月的新用户计算 10
    cursor.execute('''
        select count(distinct id) from
        (select distinct id,value from ads_user_tag_distribute
                where tag_id = 15 and value like '{}%') as reg
        where reg.value BETWEEN '{}' and '{}' and length(reg.value)=10
    '''.format(this_l1_start_month.strftime('%Y-%m-%d')[0:5],this_l1_start_month,this_l1_end_month))
    new_user_l1 = cursor.fetchall()[0][0]
    #print(this_l1_start_month.strftime('%Y-%m-%d')[5:7],"月新用户",new_user_l1)
    
    ####上月的2个月留存用户计算 9
    cursor.execute('''
        select count(distinct id) from(
            select distinct id from dws_event_2020_middle_distribute
                where app_date BETWEEN  '{}' and '{}' and
                id GLOBAL IN (
                    select distinct id from
                    (select distinct id,value from ads_user_tag_distribute
                            where tag_id = 15 and value BETWEEN '{}' and '{}' and length(value)=10
                            )
                ) 
            ) 
    '''.format(this_l1_start_month,this_end_month,this_l2_start_month,this_l2_end_month))
    this_retained_2month = cursor.fetchall()[0][0]
    #上月的2月留存率
    if(len(new_user_l2) == 0):
        Retention_rates_2month = 0
    else:
        Retention_rates_2month = this_retained_2month/len(new_user_l2) 
    #本月的目标2月留存数
    this_2month_target = round(new_user_l1*Retention_rates_2month)
    
#     print("要逐天计算留存的月份是：",this_l1_start_month,"时间范围",this_start_month,"-",this_n1_end_month)
#     print(this_l2_start_month,"上月的2个月留存用户",this_retained_2month,this_l1_start_month,"-",this_end_month)
#     print(this_l2_start_month,"上月的新用户",len(new_user_l2),"2月留存率",Retention_rates_2month)
    
    ####本日的2月内留存用户计算 
    ####eg:10月注册的在11 12月里任意天操作过都算留存 
    cursor.execute('''
        select count(distinct id) from(
            select distinct id from dws_event_2020_middle_distribute
                where app_date BETWEEN  '{}' and '{}' and
                id GLOBAL IN (
                    select distinct id from
                    (select id from ads_user_tag_distribute
                            where tag_id = 15 and value BETWEEN '{}' and '{}' and length(value)=10
                            )
                )
            )  
    '''.format(this_month,this_date,this_l1_start_month,this_l1_end_month))
    this_retained_2month = cursor.fetchall()[0][0]   
          
    
    if this_date == this_month :
        pd.DataFrame([(this_date,this_retained_2month)],columns=['ds','y']).to_csv('currrent_ret_2month.csv',index=False)
    else :
        pd.DataFrame([(this_date,this_retained_2month)],columns=['ds','y']).to_csv('currrent_ret_2month.csv',index=False,mode='a',header=False)    
        
    estimate_retention_2month(this_start_month,this_n1_end_month,this_2month_target)
    cursor.close()
    conn.close()
    if this_2month_target != 0:
        return new_user_l1,this_retained_2month,this_2month_target,round((this_retained_2month/this_2month_target)*100,2)
    else :
        return new_user_l1,this_retained_2month,this_2month_target,0    
    

In [10]:
def get_data_3month(begin_date,j,branch="所有支行"):
    client = clickhouse_driver.Client(host='chi-ftabc-clickhouse-0-0.default.svc.cluster.local', port='9000', database='ftabcch', user='ftabc', password='aihub@2020')
    conn = connect(host='chi-ftabc-clickhouse-3-0.default.svc.cluster.local', port='9000', database='ftabcch', user='ftabc', password='aihub@2021')
    cursor = conn.cursor()
    cursor.execute('''select addDays(toDate('{}'),{}),toStartOfMonth(addDays(toDate('{}'),{}))'''.format(begin_date,j,begin_date,j))
    this_date,this_month = cursor.fetchall()[0]#[0][0]-timedelta(days=1)
    #print(this_date,this_month)
    delta_month = dList.datetime.strptime(str(this_month),"%Y-%m-%d").month - dList.datetime.strptime(str(begin_date),"%Y-%m-%d").month +12*(dList.datetime.strptime(str(this_month),"%Y-%m-%d").year - dList.datetime.strptime(str(begin_date),"%Y-%m-%d").year)
    print(delta_month,this_month,this_date)
    if(delta_month>0):
        if(delta_month%3==1):
            cursor.execute('''select toStartOfMonth( addDays(toStartOfMonth(toDate('{}')),-1) )'''.format(this_month))
            this_month = cursor.fetchall()[0][0]
            print(this_month)
        if(delta_month%3==2):
            cursor.execute('''select toStartOfMonth( addDays(toStartOfMonth( addDays(toStartOfMonth(toDate('{}')),-1) ),-1) )'''.format(this_month))
            this_month = cursor.fetchall()[0][0]
            print(this_month)
                
####拿到当前月份、上月份、上上月份 的 第一天 和 最后一天
    cursor.execute('''
        select 
            '{}',
            addDays( toStartOfMonth( addDays(toDate('{}'),31) ),-1 ),
            toStartOfMonth( addDays(toStartOfMonth(toDate('{}')),-1) ),
            addDays(toStartOfMonth(toDate('{}')),-1),
            toStartOfMonth( addDays(toStartOfMonth( addDays(toStartOfMonth(toDate('{}')),-1) ),-1) ),
            addDays(toStartOfMonth( addDays(toStartOfMonth(toDate('{}')),-1) ),-1),
            toStartOfMonth( addDays(toDate('{}'),31) ),
            addDays( toStartOfMonth( addDays(toStartOfMonth( addDays(toDate('{}'),31) ),31) ),-1 ),
            toStartOfMonth( addDays(toStartOfMonth( addDays(toDate('{}'),31) ),31) ),
            addDays(toStartOfMonth( addDays(toStartOfMonth( addDays(toStartOfMonth( addDays(toDate('{}'),31) ),31) ),31)),-1)
    '''.format(this_month,this_month,this_month,this_month,this_month,this_month,this_month,this_month,this_month,this_month))
    this_two_month = cursor.fetchall()
    this_start_month = this_two_month[0][0] #2020-11-01
    this_end_month = this_two_month[0][1] #2020-11-30
    this_l1_start_month = this_two_month[0][2] #2020-10-01
    this_l1_end_month = this_two_month[0][3] #2020-10-31
    this_l2_start_month = this_two_month[0][4] # 2020-09-01
    this_l2_end_month = this_two_month[0][5] #2020-09-30
    this_n1_start_month = this_two_month[0][6] #2020-12-01
    this_n1_end_month = this_two_month[0][7]   #2020-12-31
    this_n2_start_month = this_two_month[0][8] #2020-01-01
    this_n2_end_month = this_two_month[0][9]   #2020-01-31   
    # ####拿到当前月份、上月份、上上月份
#     print(this_start_month) #2020-11-01
#     print(this_end_month) #2020-11-30
#     print(this_l1_start_month) #2020-10-01
#     print(this_l1_end_month) #2020-10-31
#     print(this_l2_start_month.strftime('%Y-%m-%d')) # 2020-09-01
#     print(this_l2_end_month.strftime('%Y-%m-%d')) #2020-09-30
    
    #print(getEveryDay(str(this_start_month),str(this_end_month)))       
                                                                                                
    
    ####上上一月的新用户计算 9
 
    cursor.execute('''
        select distinct id from
        (select distinct id,value from ads_user_tag_distribute
                where tag_id = 15 and value like '{}%') as reg
        where reg.value BETWEEN '{}' and '{}' and length(reg.value)=10
        order by value
    '''.format(this_l2_start_month.strftime('%Y-%m-%d')[0:5],this_l2_start_month,this_l2_end_month))
    new_user_l2 = cursor.fetchall()
    #print(this_l2_start_month.strftime('%Y-%m-%d')[5:7],"月新用户",len(new_user_l2))
    
    
    ####上一月的新用户计算 10
    cursor.execute('''
        select count(distinct id) from
        (select distinct id,value from ads_user_tag_distribute
                where tag_id = 15 and value like '{}%') as reg
        where reg.value BETWEEN '{}' and '{}' and length(reg.value)=10
    '''.format(this_l1_start_month.strftime('%Y-%m-%d')[0:5],this_l1_start_month,this_l1_end_month))
    new_user_l1 = cursor.fetchall()[0][0]
    #print(this_l1_start_month.strftime('%Y-%m-%d')[5:7],"月新用户",new_user_l1)
    
    ####上月的3个月留存用户计算 9
    cursor.execute('''
        select count(distinct id) from(
            select distinct id from dws_event_2020_middle_distribute
                where app_date BETWEEN  '{}' and '{}' and
                id GLOBAL IN (
                    select distinct id from
                    (select distinct id,value from ads_user_tag_distribute
                            where tag_id = 15 and value BETWEEN '{}' and '{}' and length(value)=10
                            )
                ) 
            ) 
    '''.format(this_l1_start_month,this_n1_end_month,this_l2_start_month,this_l2_end_month))
    this_retained_2month = cursor.fetchall()[0][0]
    #上月的3月留存率
    if(len(new_user_l2) == 0):
        Retention_rates_2month = 0
    else:
        Retention_rates_2month = this_retained_2month/len(new_user_l2) 
    #本月的目标3月留存数
    this_2month_target = round(new_user_l1*Retention_rates_2month)
    
#     print("要逐天计算留存的月份是：",this_l1_start_month,"时间范围",this_start_month,"-",this_n2_end_month)
#     print(this_l2_start_month,"上月的3个月留存用户",this_retained_2month,this_l1_start_month,"-",this_n1_end_month)
#     print(this_l2_start_month,"上月的新用户",len(new_user_l2),"3月留存率",Retention_rates_2month)
    
    ####本日的3月内留存用户计算 
    ####eg:10月注册的在11 12 1月里任意天操作过都算留存 
    cursor.execute('''
        select count(distinct id) from(
            select distinct id from dws_event_2020_middle_distribute
                where app_date BETWEEN  '{}' and '{}' and
                id GLOBAL IN (
                    select distinct id from
                    (select id from ads_user_tag_distribute
                            where tag_id = 15 and value BETWEEN '{}' and '{}' and length(value)=10
                            )
                )
            )  
    '''.format(this_month,this_date,this_l1_start_month,this_l1_end_month))
    this_retained_2month = cursor.fetchall()[0][0]   
          
    
    if this_date == this_month :
        pd.DataFrame([(this_date,this_retained_2month)],columns=['ds','y']).to_csv('currrent_ret_3month.csv',index=False)
    else :
        pd.DataFrame([(this_date,this_retained_2month)],columns=['ds','y']).to_csv('currrent_ret_3month.csv',index=False,mode='a',header=False)    
        
    estimate_retention_3month(this_start_month,this_n2_end_month,this_2month_target)
    cursor.close()
    conn.close()
    if this_2month_target != 0:
        return new_user_l1,this_retained_2month,this_2month_target,round((this_retained_2month/this_2month_target)*100,2)
    else :
        return new_user_l1,this_retained_2month,this_2month_target,0    
    

In [11]:
import os

name = "target_ret_2month.csv"
if os.path.exists(name):
    os.remove(name)
i = 0
i_2 = 0
i_3 = 0
tempFlag_2 = 0
tempFlag_3 = 0
begin_date='2020-11-01'
begin_date_2='2020-11-01'
begin_date_3='2020-11-01'
temp_date_2 = begin_date_2
temp_date_3 = begin_date_3


new_user_l1,this_month_mau_till_now,this_month_target_mau,complete_rate_till_now = get_data(begin_date,i)
new_user_l1_2month,this_month_mau_till_now_2month,this_month_target_mau_2month,complete_rate_till_now_2month = get_data_2month(begin_date_2,i_2)
new_user_l1_3month,this_month_mau_till_now_3month,this_month_target_mau_3month,complete_rate_till_now_3month = get_data_3month(begin_date_3,i_3)

#计算1月留存率当前的月份
now_date = dList.datetime.strptime(str(begin_date),"%Y-%m-%d")
now_date += dList.timedelta(days=i)
if now_date.month-1==0:
    now_month = 12
else:
    now_month = now_date.month-1

# #计算2月留存率当前的月份
# now_date_2 = dList.datetime.strptime(str(begin_date_2),"%Y-%m-%d")
# now_date_2 += dList.timedelta(days=i_2)
# if now_date_2.month-1==0:
#     now_month_2 = 12
# else:
#     now_month_2 = now_date_2.month-1

# #计算3月留存率当前的月份
# now_date_3 = dList.datetime.strptime(str(begin_date_3),"%Y-%m-%d")
# now_date_3 += dList.timedelta(days=i_3)
# if now_date_3.month-1==0:
#     now_month_3 = 12
# else:
#     now_month_3 = now_date_3.month-1
now_month_2 = dList.datetime.strptime(str(begin_date_2),"%Y-%m-%d").month-1
now_month_3 = dList.datetime.strptime(str(begin_date_3),"%Y-%m-%d").month-1

#1个月
this_month_mau = pd.read_csv('target_ret.csv')
all_dau_pdf=pd.read_csv('currrent_ret.csv')
all_dau_pdf.columns=['ds','y']
fig = go.Figure()
fig.add_trace(go.Scatter(x=this_month_mau['ds'], y=this_month_mau['ret'], name='本月目标ret',line=dict(color='firebrick', width=4,dash='dash')))
fig.add_trace(go.Scatter(x=all_dau_pdf['ds'], y=all_dau_pdf['y'], mode='lines',name='实际ret',line=dict(color='royalblue', width=4)))

#2个月
this_month_mau2 = pd.read_csv('target_ret_2month.csv')
all_dau_pdf2=pd.read_csv('currrent_ret_2month.csv')
all_dau_pdf2.columns=['ds','y']
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=this_month_mau2['ds'], y=this_month_mau2['ret'], name='本月目标ret',line=dict(color='firebrick', width=4,dash='dash')))
fig2.add_trace(go.Scatter(x=all_dau_pdf2['ds'], y=all_dau_pdf2['y'], mode='lines',name='实际ret',line=dict(color='royalblue', width=4)))

#3个月
this_month_mau3 = pd.read_csv('target_ret_3month.csv')
all_dau_pdf3=pd.read_csv('currrent_ret_3month.csv')
all_dau_pdf3.columns=['ds','y']
fig3 = go.Figure()
fig3.add_trace(go.Scatter(x=this_month_mau3['ds'], y=this_month_mau3['ret'], name='本月目标ret',line=dict(color='firebrick', width=4,dash='dash')))
fig3.add_trace(go.Scatter(x=all_dau_pdf3['ds'], y=all_dau_pdf3['y'], mode='lines',name='实际ret',line=dict(color='royalblue', width=4)))

app.layout = html.Div([
    dcc.Interval(id="counter", interval=2000, n_intervals=0),
    
    ######1个月
html.Div(children=[
    html.H1(id='month',children='''{}月新客的1月留存率'''.format(now_month)),
    html.Div(id='newUser',children='''新客:{}'''.format(new_user_l1)),
    html.Div(id='goal',children='''至今：0 目标：500 完成率：0%'''),
    dcc.Graph(
        id='line',
        figure=fig
    ),
], style={'textAlign': 'left','width': '30%', 'display': 'inline-block'}),
    
    ######2个月    
html.Div(children=[
    
    html.H1(id='month2',children='''{}月新客的2月留存率'''.format(now_month_2)),
    html.Div(id='newUser2',children='''新客:{}'''.format(new_user_l1_2month)),
    html.Div(id='goal2',children='''至今：0 目标：500 完成率：0%'''),
    dcc.Graph(
        id='line2',
        figure=fig2
    ),
], style={'textAlign': 'center','width': '30%', 'display': 'inline-block'}),
    
    ######3个月
html.Div(children=[
    
    html.H1(id='month3',children='''{}月新客的3月留存率'''.format(now_month_3)),
    html.Div(id='newUser3',children='''新客:{}'''.format(new_user_l1_3month)),
    html.Div(id='goal3',children='''至今：0 目标：500 完成率：0%'''),
    dcc.Graph(
        id='line3',
        figure=fig3
    ),
], style={'textAlign': 'right','width': '30%', 'display': 'inline-block'}),
    
])


@app.callback(
    [Output(component_id='month', component_property='children'),Output(component_id='newUser', component_property='children'),Output(component_id='goal', component_property='children'),Output(component_id='line',component_property='figure'),
     Output(component_id='month2', component_property='children'),Output(component_id='newUser2', component_property='children'),Output(component_id='goal2', component_property='children'),Output(component_id='line2',component_property='figure'),
     Output(component_id='month3', component_property='children'),Output(component_id='newUser3', component_property='children'),Output(component_id='goal3', component_property='children'),Output(component_id='line3',component_property='figure'),],
    [Input(component_id='counter', component_property='n_intervals'),Input('counter','n_intervals')]
)
def update_output_div(dropdown,n_intervals):
    global i
    #判断当前日期
    now_date = dList.date.today()
    new_begin_date = dList.datetime.strptime(begin_date, "%Y-%m-%d") + dList.timedelta(days = i)
    print(str(new_begin_date)[0:10],str(now_date)[0:10])
    if(str(new_begin_date)[0:10]==str(now_date)[0:10]):
        add_day_flag = 0 #"同一天"
    else:
        add_day_flag = 1 #"非同一天"
    print(add_day_flag)
    
    if(add_day_flag == 1):
        i = i+1
    new_user_l1,this_month_mau_till_now,this_month_target_mau,complete_rate_till_now = get_data(begin_date,i)
    
    global i_2
    global temp_date_2
    global tempFlag_2
    global begin_date_2
    if(add_day_flag == 1):
        i_2 = i_2+1
    a_2 = dList.datetime.strptime(begin_date_2, "%Y-%m-%d")
    a_2 = a_2 + dList.timedelta(days = i_2)
    delta_month_2 = a_2.month - dList.datetime.strptime(str(begin_date_2),"%Y-%m-%d").month++12*(a_2.year - dList.datetime.strptime(str(begin_date_2),"%Y-%m-%d").year)

    if (delta_month_2==1 and tempFlag_2==0):
        temp_date_2 = a_2.strftime('%Y-%m-%d')
        tempFlag_2 = 1
    elif(delta_month_2==2):
        i_2 = 0
        begin_date_2 = temp_date_2
        tempFlag_2 = 0    
    new_user_l1_2month,this_month_mau_till_now_2month,this_month_target_mau_2month,complete_rate_till_now_2month = get_data_2month(begin_date_2,i_2)
    
    global i_3
    global temp_date_3
    global tempFlag_3
    global begin_date_3
    if(add_day_flag == 1):    
        i_3 = i_3+1
    a_3 = dList.datetime.strptime(begin_date_3, "%Y-%m-%d")
    a_3 = a_3 + dList.timedelta(days = i_3)
    delta_month_3 = a_3.month - dList.datetime.strptime(str(begin_date_3),"%Y-%m-%d").month++12*(a_3.year - dList.datetime.strptime(str(begin_date_3),"%Y-%m-%d").year)

    if (delta_month_3==1 and tempFlag_3==0):
        temp_date_3 = a_3.strftime('%Y-%m-%d')
        tempFlag_3 = 1
    elif(delta_month_3==3):
        i_3 = 0
        begin_date_3 = temp_date_3
        tempFlag_3 = 0        
    new_user_l1_3month,this_month_mau_till_now_3month,this_month_target_mau_3month,complete_rate_till_now_3month = get_data_3month(begin_date_3,i_3)
    #1个月
    this_month_mau = pd.read_csv('target_ret.csv')
    all_dau_pdf=pd.read_csv('currrent_ret.csv')
    all_dau_pdf.columns=['ds','y']
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=this_month_mau['ds'], y=this_month_mau['ret'], name='本月目标ret',line=dict(color='firebrick', width=4,dash='dash')))
    fig.add_trace(go.Scatter(x=all_dau_pdf['ds'], y=all_dau_pdf['y'], mode='lines',name='实际ret',line=dict(color='royalblue', width=4)))

    #2个月
    this_month_mau2 = pd.read_csv('target_ret_2month.csv')
    all_dau_pdf2=pd.read_csv('currrent_ret_2month.csv')
    all_dau_pdf2.columns=['ds','y']
    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(x=this_month_mau2['ds'], y=this_month_mau2['ret'], name='本月目标ret',line=dict(color='firebrick', width=4,dash='dash')))
    fig2.add_trace(go.Scatter(x=all_dau_pdf2['ds'], y=all_dau_pdf2['y'], mode='lines',name='实际ret',line=dict(color='royalblue', width=4)))

    #3个月
    this_month_mau3 = pd.read_csv('target_ret_3month.csv')
    all_dau_pdf3=pd.read_csv('currrent_ret_3month.csv')
    all_dau_pdf3.columns=['ds','y']
    fig3 = go.Figure()
    fig3.add_trace(go.Scatter(x=this_month_mau3['ds'], y=this_month_mau3['ret'], name='本月目标ret',line=dict(color='firebrick', width=4,dash='dash')))
    fig3.add_trace(go.Scatter(x=all_dau_pdf3['ds'], y=all_dau_pdf3['y'], mode='lines',name='实际ret',line=dict(color='royalblue', width=4)))
    
    #计算1月留存率当前的月份
    now_date = dList.datetime.strptime(str(begin_date),"%Y-%m-%d")
    now_date += dList.timedelta(days=i)
    if now_date.month-1==0:
        now_month = 12
    else:
        now_month = now_date.month-1

#     #计算2月留存率当前的月份
#     now_date_2 = dList.datetime.strptime(str(begin_date_2),"%Y-%m-%d")
#     now_date_2 += dList.timedelta(days=i_2)
#     if now_date_2.month-1==0:
#         now_month_2 = 12
#     else:
#         now_month_2 = now_date_2.month-1

#     #计算3月留存率当前的月份
#     now_date_3 = dList.datetime.strptime(str(begin_date_3),"%Y-%m-%d")
#     now_date_3 += dList.timedelta(days=i_3)
#     if now_date_3.month-1==0:
#         now_month_3 = 12
#     else:
#         now_month_3 = now_date_3.month-1
    now_month_2 = dList.datetime.strptime(str(begin_date_2),"%Y-%m-%d").month-1
    now_month_3 = dList.datetime.strptime(str(begin_date_3),"%Y-%m-%d").month-1
    return '''{}月新客的1月留存率'''.format(now_month),'''新客:{}'''.format(new_user_l1),'''至今：{} 目标：{} 完成率：{}%'''.format(this_month_mau_till_now,this_month_target_mau,complete_rate_till_now),fig,'''{}月新客的2月留存率'''.format(now_month_2),'''新客:{}'''.format(new_user_l1_2month),'''至今：{} 目标：{} 完成率：{}%'''.format(this_month_mau_till_now_2month,this_month_target_mau_2month,complete_rate_till_now_2month),fig2,'''{}月新客的3月留存率'''.format(now_month_3),'''新客:{}'''.format(new_user_l1_3month),'''至今：{} 目标：{} 完成率：{}%'''.format(this_month_mau_till_now_3month,this_month_target_mau_3month,complete_rate_till_now_3month),fig3
if __name__ == '__main__':
    app.run_server(debug=True,mode="inline")

0 2020-11-01
0 2020-11-01 2020-11-01


0 2020-11-02
0 2020-11-01 2020-11-02
0 2020-11-03
0 2020-11-01 2020-11-03
0 2020-11-04
0 2020-11-01 2020-11-04
0 2020-11-05
0 2020-11-01 2020-11-05
0 2020-11-06
0 2020-11-01 2020-11-06
0 2020-11-07
0 2020-11-01 2020-11-07
0 2020-11-08
0 2020-11-01 2020-11-08
0 2020-11-09
0 2020-11-01 2020-11-09
0 2020-11-10
0 2020-11-11
0 2020-11-01 2020-11-10
0 2020-11-01 2020-11-11
0 2020-11-12
0 2020-11-01 2020-11-12
0 2020-11-13
0 2020-11-01 2020-11-13
0 2020-11-14
0 2020-11-01 2020-11-14
0 2020-11-15
0 2020-11-01 2020-11-15
0 2020-11-16
0 2020-11-01 2020-11-16
0 2020-11-17
0 2020-11-01 2020-11-17
0 2020-11-18
0 2020-11-01 2020-11-18
0 2020-11-19
0 2020-11-01 2020-11-19
0 2020-11-20
0 2020-11-01 2020-11-20
0 2020-11-21
0 2020-11-01 2020-11-21
0 2020-11-22
0 2020-11-01 2020-11-22
0 2020-11-23
0 2020-11-01 2020-11-23
0 2020-11-24
0 2020-11-01 2020-11-24
0 2020-11-25
0 2020-11-01 2020-11-25
0 2020-11-26
0 2020-11-01 2020-11-26
0 2020-11-27
0 2020-11-01 2020-11-27
0 2020-11-28
0 2020-11-01 2020-11-28
0

In [13]:
begin_date='2021-03-23'
now_date = dList.date.today()
new_begin_date = dList.datetime.strptime(begin_date, "%Y-%m-%d") + dList.timedelta(days = 1)
print(str(new_begin_date)[0:10],str(now_date)[0:10])
if(str(new_begin_date)[0:10]==str(now_date)[0:10]):
    add_day_flag = 0
else:
    add_day_flag = 1
print(add_day_flag)

2021-03-24 2021-03-24
0
